# Wide and Deep Networks for Credit Score Classification

By: Joe, Sellett, Haiyan Cai, and Cole Wagner

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import FeatureSpace


In [2]:
credit_df = pd.read_csv("credit_score_cleaned.csv")

## Data Preparation

### Drop Unnecessary Columns

Before proceeding with the modeling phase of this project, we will remove the following variables: customer_id, name, ssn, and type_of_loan. The customer_id field is being excluded because we already have a more robust unique identifier, id, which will serve as our primary reference for credit score reports. Similarly, name and ssn offer no predictive value and are being dropped to maintain data privacy and reduce dimensionality. Each of these variables contains approximately 8,000–12,000 unique values, whereas id contains over 96,000. Lastly, type_of_loan is being excluded for its high number of categories (50+), which would introduce unnecessary complexity. Instead, we will rely on the credit_mix variable, which summarizes loan diversity in a more manageable form as a category with only 3 unique values (standard, good, and bad).

In [3]:
credit_df = credit_df.drop(
    columns=["customer_id", "name", "ssn", "type_of_loan"]
)

### Create Feature Space for Preprocessing

In [4]:
def create_dataset_from_dataframe(
    x_input: pd.DataFrame, y_input: pd.Series, batch_size: int
) -> tf.data.Dataset:
    """Convert a pandas dataframe to a TensorFlow Dataset.

    Parameters
    ----------
    x_input : pd.DataFrame
        The input pandas dataframe containing the features.
    y_input : pd.Series
        The input pandas series containing the labels.
    batch_size : int
        The number of rows per batch in the TensorFlow Dataset.

    Returns
    -------
    tf.data.Dataset
        A TensorFlow Dataset object created from the input dataframe.

    """
    df_dict = {
        key: value.to_numpy()[:, np.newaxis]
        for key, value in x_input.items()
    }

    tf_ds = tf.data.Dataset.from_tensor_slices((dict(df_dict), y_input))
    tf_ds = tf_ds.batch(batch_size)
    return tf_ds.prefetch(batch_size)


In [5]:
# Sample schema based on the dataframe info
categorical_features = [
    "month",
    "occupation",
    "credit_mix",
    "payment_of_min_amount",
    "payment_behaviour",
]
numeric_features = [
    "age",
    "annual_income",
    "monthly_inhand_salary",
    "credit_history_age",
    "total_emi_per_month",
    "num_bank_accounts",
    "num_credit_card",
    "interest_rate",
    "num_of_loan",
    "delay_from_due_date",
    "num_of_delayed_payment",
    "changed_credit_limit",
    "num_credit_inquiries",
    "outstanding_debt",
    "credit_utilization_ratio",
    "amount_invested_monthly",
    "monthly_balance",
]

# Define feature configs
feature_space = FeatureSpace(
    features={
        **{
            name: FeatureSpace.string_categorical(num_oov_indices=0)
            for name in categorical_features
        },
        **{
            name: FeatureSpace.float_normalized()
            for name in numeric_features
        },
    },
    crosses=[
        ("occupation", "credit_mix"),
        ("payment_of_min_amount", "payment_behaviour"),
    ],
    output_mode="concat",
)

### Cross-Product Feature Justification

First, we created a cross-product feature between `occupation` and `credit_mix`. This combination allows us to capture differences in credit behavior across various professional backgrounds. For example, a neurosurgeon with a bad credit mix may exhibit very different financial behavior compared to an unemployed individual with the same credit mix. While each variable on its own may offer limited insight, their combination provides a more nuanced understanding of how occupation and credit diversity interact.

Another cross-product feature we created combines `payment_of_min_amount` and `payment_behavior`. The `payment_of_min_amount` variable is a binary indicator showing whether an individual made only the minimum payment on their debt for that month. In contrast, `payment_behavior` provides a broader description of a person’s spending and repayment patterns, such as “low spent, high payments” or “high spent, medium payments.” Since these two variables are closely related, their combination may help the model better capture nuanced repayment behaviors and improve its ability to distinguish between risk profiles.

### Performance Metric Justification

Given the nature of our project, it’s important to evaluate our model using multiple metrics rather than relying solely on accuracy. In credit risk classification, false predictions carry different levels of business risk. For example, if a high-risk individual is incorrectly classified as low-risk, the company may absorb the financial loss from a bad loan. This makes recall especially important, as it tells us how well the model identifies actual high-risk cases and helps minimize false negatives. At the same time, precision matters because it reflects how accurate our high-risk predictions are, which ensures we don’t wrongly classify low-risk individuals as high-risk. A high recall means we’re catching most of the truly risky borrowers, while a high precision score means we’re correctly labeling them. Since both metrics are critical and often trade off against each other, we focus on the F1 score, which represents the harmonic mean of precision and recall. The F1 score gives us a more balanced and realistic measure of performance, especially in a setting where both catching risky borrowers and avoiding false alarms are essential to the business.

### Data Splitting

We have chosen to use a standard 80/20 train-test split for dividing our dataset. Given the size of our data (approximately 100,000 observations) we believe this approach is justified and will provide a reliable estimate of model performance. If our dataset were significantly smaller (around 1,000 observations), we might opt for 10-fold cross-validation to obtain a more stable and generalized result. Additionally, the 80/20 split offers a clear advantage in terms of computational efficiency. While 10-fold cross-validation could yield a marginal improvement in performance estimates, it would come at a considerable computational cost that is unnecessary given the scale of our data.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    credit_df.drop(columns=["credit_score", "id"]),
    credit_df["credit_score"],
    test_size=0.2,
    random_state=7324,
    stratify=credit_df["credit_score"],
)

In [7]:
# Convert data to TensorFlow Datasets
train_ds = create_dataset_from_dataframe(x_train, y_train, batch_size=32)
test_ds = create_dataset_from_dataframe(x_test, y_test, batch_size=32)

In [8]:
# Apply feature space to datasets
train_ds_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_no_labels)
processed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)
processed_train_ds = processed_train_ds.prefetch(tf.data.AUTOTUNE)

test_ds_no_labels = test_ds.map(lambda x, _: x)
feature_space.adapt(test_ds_no_labels)
processed_test_ds = test_ds.map(
    lambda x, y: (feature_space(x), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)
processed_test_ds = processed_test_ds.prefetch(tf.data.AUTOTUNE)


2025-04-12 15:19:44.979140: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-12 15:19:45.620501: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-12 15:19:46.942109: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-12 15:19:51.684273: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-12 15:20:02.647269: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-12 15:20:13.460720: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
